In [12]:
start = '''\
# import needed libraries 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
# collect data
df = pd.read_csv('callisto_example_data.csv')
df.sent_date = pd.to_datetime(df.sent_date)
df['sent_month'] = df.sent_date.dt.month
df['response_time_hours'] = df.response_time / 3600

# custom percentile function to assist
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

# custom function to help calculate percentage of a particular status
def status_to_percent(status):
    def status_to_percent_(x):
        return sum(x == status) / len(x)
    # take out white space and make lower case
    stripped = status.replace(" ", "")
    stripped = stripped.lower()
    status_to_percent_.__name__ = '%s_percentage' % stripped
    return status_to_percent_

df_by_month = df.groupby('sent_month').agg({
    'hotel':['count'],
    'room_night_count':[percentile(10), percentile(90), 'median'],
    'attendee_count':[percentile(10), percentile(90), 'median'],
    'status': [
        status_to_percent('Declined'),
        status_to_percent('Turned Down'),
        status_to_percent('Submitted Proposal'),
        status_to_percent('Awarded'),
        status_to_percent('Withdrawn'),
        status_to_percent('Awaiting Proposal'),
        status_to_percent('No Response'),
        status_to_percent('Cancelled')
    ]})'''

code1 = '''temp = df[df.hotel == '%s']'''

code2 = '''\
sns.distplot(temp.room_night_count, kde=False, bins=20 )
plt.title('Historgram of Room Count')
plt.xlabel('Room Count')
plt.ylabel('Count')
'''

code3 = '''\
sns.distplot(temp.room_night_count, kde=False, bins=20)
plt.title('Historgram of Attendee Count')
plt.xlabel('Attendee Count')
plt.ylabel('Count')'''

code4 = '''\
sns.distplot(temp.response_time_hours, kde=False, bins=20)
plt.title('Historgram of respone_time')
plt.xlabel('Respone Time Hours')
plt.ylabel('Count')'''

code5 = '''\
sns.boxplot(data=temp, x='sent_month', y='room_night_count')
plt.title('Boxplot of Room Count by Month')
plt.xlabel('Sent Month')
plt.ylabel('Room Count')'''

code6 = '''\
sns.boxplot(data=temp, x='sent_month', y='attendee_count')
plt.title('Boxplot of Attendee Count by Month')
plt.xlabel('Sent Month')
plt.ylabel('Attendee Count')'''

code7 = '''\
sns.boxplot(data=temp, x='sent_month', y='response_time_hours')
plt.title('Boxplot of Room Count by Month')
plt.xlabel('Sent Month')
plt.ylabel('Response Time Hours')'''

In [13]:
import nbformat as nbf
import pandas as pd

df = pd.read_csv('callisto_example_data.csv')

nb = nbf.v4.new_notebook()
# nb.cells is a list that we just append to to create cells
nb.cells.append(nbf.v4.new_markdown_cell('# This is a generated notebook'))
nb.cells.append(nbf.v4.new_code_cell(start))
for hotel in df.hotel.unique():
    nb.cells.append(nbf.v4.new_markdown_cell('This is for %s' % hotel))
    nb.cells.append(nbf.v4.new_code_cell(code1 % hotel))
    nb.cells.append(nbf.v4.new_code_cell(code2))
    nb.cells.append(nbf.v4.new_code_cell(code3))
    nb.cells.append(nbf.v4.new_code_cell(code4))
    nb.cells.append(nbf.v4.new_code_cell(code5))
    nb.cells.append(nbf.v4.new_code_cell(code6))
    nb.cells.append(nbf.v4.new_code_cell(code7))
    

with open('hotel_report_generated.ipynb', 'wt') as f:
    nbf.write(nb, f)